# FPL API Request

This script requests all player histories from the FPL API, and saves the result as a BigQuery dataset. 


In [ ]:
!pip install google-cloud-bigquery

import requests
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account


With the libraries we've installed and loaded, we can request the data we want from the FPL API.

In [ ]:
# Get all player IDs.
players_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(players_url)
data = response.json()
players = pd.DataFrame(data["elements"])

positions = players[["code", "element_type"]]

# Request historic data for each player.
for element_id in players["id"].unique():
    url = f'https://fantasy.premierleague.com/api/element-summary/{element_id}/'
    r = requests.get(url)
    r = r.json()
    player_history = pd.DataFrame(r['history_past'])
    if element_id == players["id"].unique()[0]:
        all_history = player_history
    else:
        all_history = pd.concat([all_history, player_history])
        
# Reformat dataframes as list of dictionaries so BQ can read it.
all_history_dict = all_history.to_dict('records')
positions_dict = positions.to_dict('records')


Now let's set up a BigQuery table to hold the data, and allow us to process it easily.

In [ ]:
# Set up BigQuery client using service account credentials.
credentials = service_account.Credentials.from_service_account_file(
    "bf-fpl-pred-080723-72e03a26c091.json",
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

In [ ]:
# Create the table if it doesn't exist.
schema_historic = [
    bigquery.SchemaField('season_name', 'STRING'),
    bigquery.SchemaField('element_code', 'INTEGER'),
    bigquery.SchemaField('start_cost', 'INTEGER'),
    bigquery.SchemaField('end_cost', 'INTEGER'),
    bigquery.SchemaField('total_points', 'INTEGER'),
    bigquery.SchemaField('minutes', 'INTEGER'),
    bigquery.SchemaField('goals_scored', 'INTEGER'),
    bigquery.SchemaField('assists', 'INTEGER'),
    bigquery.SchemaField('clean_sheets', 'INTEGER'),
    bigquery.SchemaField('goals_conceded', 'INTEGER'),
    bigquery.SchemaField('own_goals', 'INTEGER'),
    bigquery.SchemaField('penalties_saved', 'INTEGER'),
    bigquery.SchemaField('penalties_missed', 'INTEGER'),
    bigquery.SchemaField('yellow_cards', 'INTEGER'),
    bigquery.SchemaField('red_cards', 'INTEGER'),
    bigquery.SchemaField('saves', 'INTEGER'),
    bigquery.SchemaField('bonus', 'INTEGER'),
    bigquery.SchemaField('bps', 'INTEGER'),
    bigquery.SchemaField('influence', 'FLOAT'),
    bigquery.SchemaField('creativity', 'FLOAT'),
    bigquery.SchemaField('threat', 'FLOAT'),
    bigquery.SchemaField('ict_index', 'FLOAT'),
    bigquery.SchemaField('starts', 'INTEGER'),
    bigquery.SchemaField('expected_goals', 'FLOAT'),
    bigquery.SchemaField('expected_assists', 'FLOAT'),
    bigquery.SchemaField('expected_goal_involvements', 'FLOAT'),
    bigquery.SchemaField('expected_goals_conceded', 'FLOAT'),
    bigquery.SchemaField('id', 'INTEGER')
]

schema_positions = [
    bigquery.SchemaField('code', 'INTEGER'),
    bigquery.SchemaField('element_type', 'INTEGER')
]

dataset_id = "fpl_historic_data"
# Create historic data table.
table_id_historic = "raw_historic_data"
dataset_ref = client.dataset(dataset_id)
table_ref_historic = dataset_ref.table(table_id_historic)
table_historic = bigquery.Table(table_ref_historic, schema=schema_historic)
table_historic = client.create_table(table_historic, exists_ok=True)

# Create positions table.
table_id_positions = "player_positions"
table_ref_positions = dataset_ref.table(table_id_positions)
table_positions = bigquery.Table(table_ref_positions, schema=schema_positions)
table_positions = client.create_table(table_historic, exists_ok=True)



In [ ]:
# Insert data into the table, overwriting any existing data.
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE 
job = client.load_table_from_json(all_history_dict, table_ref_historic, job_config=job_config)
job.result()

print(f'Data uploaded to BigQuery table {table_id_historic} in dataset {dataset_id}')

job = client.load_table_from_json(positions_dict, table_ref_positions, job_config=job_config)
job.result()

print(f'Data uploaded to BigQuery table {table_id_positions} in dataset {dataset_id}')

Data uploaded to BigQuery table raw_historic_data in dataset fpl_historic_data
Data uploaded to BigQuery table player_positions in dataset fpl_historic_data
